In [1]:
!pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
import transformers
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

In [3]:
# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # Set pad token

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [14]:
# Load a poetry dataset from Hugging Face
dataset = load_dataset("biglam/gutenberg-poetry-corpus")


In [18]:
dataset.keys()

dict_keys(['train'])

In [19]:
if "train" in dataset:
    dataset = dataset["train"]
else:
    raise KeyError("Dataset does not contain a 'train' split")

In [20]:
# Shuffle and select 50k samples for faster training
dataset = dataset.shuffle(seed=42).select(range(min(50000, len(dataset))))

In [21]:
# Remove unnecessary columns
dataset = dataset.remove_columns(["gutenberg_id"])

In [22]:
# Split dataset into train and eval
split_dataset = dataset.train_test_split(test_size=0.1)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]


In [23]:
def tokenize_function(examples):
    tokenized = tokenizer(examples["line"], padding="max_length", truncation=True, max_length=128)
    tokenized["labels"] = tokenized["input_ids"].copy()  # Add labels for causal LM
    return tokenized

In [24]:
# Tokenize datasets
tokenized_train_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_eval_dataset = eval_dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/45000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [25]:
# In cell 29, include 'labels' in the columns:

tokenized_train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_eval_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

In [26]:
# Load GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [27]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-poetry",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=200,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [28]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

In [29]:
# Train the model
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: njnavy9852 (njnavy9852-navneet-and-avneet) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,0.376900,0.372907
2,0.344300,0.371165
3,0.328200,0.373558


TrainOutput(global_step=16875, training_loss=0.3562161404079861, metrics={'train_runtime': 5622.3395, 'train_samples_per_second': 24.011, 'train_steps_per_second': 3.001, 'total_flos': 8818606080000000.0, 'train_loss': 0.3562161404079861, 'epoch': 3.0})

In [31]:
# Save the fine-tuned model
model.save_pretrained("/content/drive/MyDrive/Colab Notebooks/gpt2-poetry")
tokenizer.save_pretrained("/content/drive/MyDrive/Colab Notebooks/gpt2-poetry")

print("Fine-tuning complete! Model saved at ./gpt2-poetry")

Fine-tuning complete! Model saved at ./gpt2-poetry


In [41]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load the fine-tuned model
model = GPT2LMHeadModel.from_pretrained("./gpt2-poetry")
tokenizer = GPT2Tokenizer.from_pretrained("./gpt2-poetry")

def generate_poetry(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    output = model.generate(
        **inputs,
        max_new_tokens=150,  # Ensure more tokens are generated
        do_sample=True,
        temperature=1.2,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.05,  # Less strict repetition penalty
        pad_token_id=tokenizer.eos_token_id  # Explicitly set pad token ID
    )
    return tokenizer.decode(output[0], skip_special_tokens=True)

print(generate_poetry("Once upon a midnight dreary,"))



Once upon a midnight dreary,


In [42]:
trainer.state.log_history

[{'loss': 0.5199,
  'grad_norm': 0.8728024363517761,
  'learning_rate': 4.940740740740741e-05,
  'epoch': 0.035555555555555556,
  'step': 200},
 {'loss': 0.4022,
  'grad_norm': 0.5241988301277161,
  'learning_rate': 4.881481481481482e-05,
  'epoch': 0.07111111111111111,
  'step': 400},
 {'loss': 0.4017,
  'grad_norm': 0.4632166922092438,
  'learning_rate': 4.8222222222222225e-05,
  'epoch': 0.10666666666666667,
  'step': 600},
 {'loss': 0.3969,
  'grad_norm': 0.5035160779953003,
  'learning_rate': 4.762962962962963e-05,
  'epoch': 0.14222222222222222,
  'step': 800},
 {'loss': 0.3898,
  'grad_norm': 0.5122094750404358,
  'learning_rate': 4.703703703703704e-05,
  'epoch': 0.17777777777777778,
  'step': 1000},
 {'loss': 0.3853,
  'grad_norm': 0.60881507396698,
  'learning_rate': 4.644444444444445e-05,
  'epoch': 0.21333333333333335,
  'step': 1200},
 {'loss': 0.3914,
  'grad_norm': 0.4130977988243103,
  'learning_rate': 4.585185185185185e-05,
  'epoch': 0.24888888888888888,
  'step': 140